In [4]:
'''
Manual evaluation of the data based on the topics obtained from the analysis

list 1: top 3 specific terms
list 2: top 5 general terms

for a document to qualify: must include at least 1 word from list 1 AND 1 word from list 2.
each document is a list, if the list contains 1 or more words from list 1 and list 2 then add the document?

'''

'\nManual evaluation of the data based on the topics obtained from the analysis\n\nlist 1: top 3 specific terms\nlist 2: top 5 general terms\n\nfor a document to qualify: must include at least 1 word from list 1 AND 1 word from list 2.\neach document is a list, if the list contains 1 or more words from list 1 and list 2 then add the document?\n\n'

In [5]:
import pickle
import re
import statistics
import nltk
import string
from nltk.stem.porter import *
from nltk.corpus import stopwords
import os
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from __future__ import print_function

from nltk.stem.porter import PorterStemmer
import scipy.sparse
import numpy as np
from pandas import DataFrame 
from random import shuffle   

import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt


# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

print("imported packages")

C:\Users\NHari\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


imported packages


In [6]:
with open(('FILE'), 'rb') as fp:
            data_lemmatized_v2 = pickle.load(fp)

In [7]:
print(data_lemmatized_v2[1])

['hope', 'china', 'democracy', 'sick', 'nobel_peace', 'laureate', 'help', 'write', 'petition', 'democratic', 'china', 'sideline', 'prison', 'parole', 'cancer', 'treatment']


In [8]:
'''
USES LEMMATIZED SENTENCES
*Topic 2: environmental/health politics
health, today, federal, government, official, state, agency, plan, law, worker, say, 
bill, asbestos, plant, would, congress, public, pay, Washington, environmental, 
tobacco, insurance, ban, propose, suit, nation, industry, chemical, nuclear, price

    //search for documents containing: 
'''
search_2 = ["worker", "asbestos", "congress"] #top 3 unique
search_2b = ["health", "today", "federal", "government", "official"] #top 5 general
w, h = 2, 9
search_2_list = [[0 for x in range(w)] for y in range(h)]
search_2_list[0][0] = "Term"
search_2_list[0][1] = "Term Frequency"
for i in range(len(search_2)):
    search_2_list[i+1][0] = search_2[i]
for i in range(len(search_2b)):
    search_2_list[i+4][0] = search_2b[i]

print("unique terms: ", search_2)
print("general terms: ", search_2b)
topic_documents_2 = []
for i in range(len(data_lemmatized_v2)):
    score_a = []
    score_b = []
    for j in range(len(data_lemmatized_v2[i])):
        if data_lemmatized_v2[i][j] in search_2:
            score_a.append(str(data_lemmatized_v2[i][j]))
        if data_lemmatized_v2[i][j] in search_2b:
            score_b.append(str(data_lemmatized_v2[i][j]))
    if len(score_a) > 0 and len(score_b) > 0:
        topic_documents_2.append(data_lemmatized_v2[i])
        for t in range(len(score_a)): 
            for k in range(len(search_2_list)):
                if search_2_list[k][0] in score_a[t]:
                    search_2_list[k][1] = search_2_list[k][1] + 1
        for v in range(len(score_b)):
            for b in range(len(search_2_list)):
                if search_2_list[b][0] in score_b[v]:
                    search_2_list[b][1] = search_2_list[b][1] + 1
    
print("amount of documents: ",len(topic_documents_2))
print(search_2_list)


unique terms:  ['worker', 'asbestos', 'congress']
general terms:  ['health', 'today', 'federal', 'government', 'official']
amount of documents:  760
[['Term', 'Term Frequency'], ['worker', 366], ['asbestos', 405], ['congress', 332], ['health', 389], ['today', 357], ['federal', 208], ['government', 136], ['official', 146]]


In [9]:
print(search_2_list)

[['Term', 'Term Frequency'], ['worker', 366], ['asbestos', 405], ['congress', 332], ['health', 389], ['today', 357], ['federal', 208], ['government', 136], ['official', 146]]


In [10]:
'''
USES LEMMATIZED SENTENCES
for topic 3: research
study, cancer, find, scientist, research, risk, aid, researcher, report, disease, may, new, human, link, 
test, gene, cell, rate, high, blood, use, lead, people, accord, evidence, death, expert, scientific, animal, health
'''
search_3 = ["study", "scientist", "risk"] #top 3 unique
search_3b = ["cancer", "find", "research", "aid", "report"] #top 5 general
print("unique terms: ", search_3)
print("general terms: ", search_3b)
w, h = 2, 9
search_3_list = [[0 for x in range(w)] for y in range(h)]
search_3_list[0][0] = "Term"
search_3_list[0][1] = "Term Frequency"
for i in range(len(search_3)):
    search_3_list[i+1][0] = search_3[i]
for i in range(len(search_3b)):
    search_3_list[i+4][0] = search_3b[i]
topic_documents_3 = []
for i in range(len(data_lemmatized_v2)):
    score_a = []
    score_b = []
    for j in range(len(data_lemmatized_v2[i])):
        if data_lemmatized_v2[i][j] in search_3:
            score_a.append(data_lemmatized_v2[i][j])
        if data_lemmatized_v2[i][j] in search_3b:
            score_b.append(data_lemmatized_v2[i][j])
    if len(score_a) > 0 and len(score_b) > 0:
        topic_documents_3.append(data_lemmatized_v2[i])
        for t in range(len(score_a)): 
            for k in range(len(search_3_list)):
                if search_3_list[k][0] in score_a[t]:
                    search_3_list[k][1] = search_3_list[k][1] + 1
        for v in range(len(score_b)):
            for b in range(len(search_3_list)):
                if search_3_list[b][0] in score_b[v]:
                    search_3_list[b][1] = search_3_list[b][1] + 1
    
print("amount of documents: ",len(topic_documents_3))
print(search_3_list)

unique terms:  ['study', 'scientist', 'risk']
general terms:  ['cancer', 'find', 'research', 'aid', 'report']
amount of documents:  2848
[['Term', 'Term Frequency'], ['study', 2232], ['scientist', 1076], ['risk', 927], ['cancer', 3126], ['find', 1107], ['research', 598], ['aid', 359], ['report', 707]]


In [11]:
'''
USES LEMMATIZED SENTENCES
Topic 4: health care
cancer, doctor, drug, patient, breast, treatment, medical, say, surgery, treat, woman, care, medicine, hospital, center, 
prostate, patent, institute, physician, approval, pain, today, therapy, receive, undergo, tumor, test, pill, health, remove

    //search for documents containing: care, program, cost
'''
search_4 = ["doctor", "patient", "breast"] #top 3 unique
search_4b = ["cancer", "drug", "medical", "say", "woman"] #top 5 general
print("unique terms: ", search_4)
print("general terms: ", search_4b)
topic_documents_4 = []

w, h = 2, 9
search_4_list = [[0 for x in range(w)] for y in range(h)]
search_4_list[0][0] = "Term"
search_4_list[0][1] = "Term Frequency"
for i in range(len(search_4)):
    search_4_list[i+1][0] = search_4[i]
for i in range(len(search_4b)):
    search_4_list[i+4][0] = search_4b[i]
    

for i in range(len(data_lemmatized_v2)):
    score_a = []
    score_b = []
    for j in range(len(data_lemmatized_v2[i])):
        if data_lemmatized_v2[i][j] in search_4:
            score_a.append(data_lemmatized_v2[i][j])
        if data_lemmatized_v2[i][j] in search_4b:
            score_b.append(data_lemmatized_v2[i][j])
    if len(score_a) > 0 and len(score_b) > 0:
        topic_documents_4.append(data_lemmatized_v2[i])
        for t in range(len(score_a)): 
            for k in range(len(search_4_list)):
                if search_4_list[k][0] in score_a[t]:
                    search_4_list[k][1] = search_4_list[k][1] + 1
        for v in range(len(score_b)):
            for b in range(len(search_4_list)):
                if search_4_list[b][0] in score_b[v]:
                    search_4_list[b][1] = search_4_list[b][1] + 1
    
print("amount of documents: ",len(topic_documents_4))
print(search_4_list)


unique terms:  ['doctor', 'patient', 'breast']
general terms:  ['cancer', 'drug', 'medical', 'say', 'woman']
amount of documents:  3370
[['Term', 'Term Frequency'], ['doctor', 1737], ['patient', 1664], ['breast', 1664], ['cancer', 3599], ['drug', 1121], ['medical', 757], ['say', 1308], ['woman', 887]]


In [12]:
'''
USES LEMMATIZED SENTENCES
for topic 5: American Politics
new, york, city, jersey, state, campaign, senator, school, republican, fund, tax, site, political, award, mayor, election, 
governor, theater, party, association, senate, public, candidate, term, office, dance, self, protest, marta_stewart, albany
'''
search_5 = ["city", "jersey", "campaign"] #top 3 unique
search_5b = ["new", "york", "state", "school", "political"] #top 5 general
print("unique terms: ", search_5)
print("general terms: ", search_5b)
topic_documents_5 = []

w, h = 2, 9
search_5_list = [[0 for x in range(w)] for y in range(h)]
search_5_list[0][0] = "Term"
search_5_list[0][1] = "Term Frequency"
for i in range(len(search_5)):
    search_5_list[i+1][0] = search_5[i]
for i in range(len(search_5b)):
    search_5_list[i+4][0] = search_5b[i]

for i in range(len(data_lemmatized_v2)):
    score_a = []
    score_b = []
    for j in range(len(data_lemmatized_v2[i])):
        if data_lemmatized_v2[i][j] in search_5:
            score_a.append(data_lemmatized_v2[i][j])
        if data_lemmatized_v2[i][j] in search_5b:
            score_b.append(data_lemmatized_v2[i][j])
    if len(score_a) > 0 and len(score_b) > 0:
        topic_documents_5.append(data_lemmatized_v2[i])
        for t in range(len(score_a)): 
            for k in range(len(search_5_list)):
                if search_5_list[k][0] in score_a[t]:
                    search_5_list[k][1] = search_5_list[k][1] + 1
        for v in range(len(score_b)):
            for b in range(len(search_5_list)):
                if search_5_list[b][0] in score_b[v]:
                    search_5_list[b][1] = search_5_list[b][1] + 1
    
print("amount of documents: ",len(topic_documents_5))
print(search_5_list)

unique terms:  ['city', 'jersey', 'campaign']
general terms:  ['new', 'york', 'state', 'school', 'political']
amount of documents:  2184
[['Term', 'Term Frequency'], ['city', 1528], ['jersey', 1015], ['campaign', 316], ['new', 2614], ['york', 1218], ['state', 638], ['school', 257], ['political', 102]]


In [13]:
'''
USES LEMMATIZED SENTENCES
Topic 6: mortality
die, old, year, former, hospital, cancer, yesterday, dead, home, president, wife, retire, chief, police, years_old_lived, kill, 
edward, lawyer, professor, peter, death, ford, sentence, university, officer, testify, george, murder, chairman, brooklyn

    //FORMERLY search for documents containing: environmental, government, safety
    //search for: chemical, judge, ban, safety, pesticide
'''
search_6 = ["die", "dead", "retire"] #top 3 unique
search_6b = ["old", "year", "former", "hospital", "cancer"] #top 5 general
print("unique terms: ", search_6)
print("general terms: ", search_6b)
topic_documents_6 = []

w, h = 2, 9
search_6_list = [[0 for x in range(w)] for y in range(h)]
search_6_list[0][0] = "Term"
search_6_list[0][1] = "Term Frequency"
for i in range(len(search_6)):
    search_6_list[i+1][0] = search_6[i]
for i in range(len(search_6b)):
    search_6_list[i+4][0] = search_6b[i]

for i in range(len(data_lemmatized_v2)):
    score_a = []
    score_b = []
    for j in range(len(data_lemmatized_v2[i])):
        if data_lemmatized_v2[i][j] in search_6:
            score_a.append(data_lemmatized_v2[i][j])
        if data_lemmatized_v2[i][j] in search_6b:
            score_b.append(data_lemmatized_v2[i][j])
    if len(score_a) > 0 and len(score_b) > 0:
        topic_documents_6.append(data_lemmatized_v2[i])
        for t in range(len(score_a)): 
            for k in range(len(search_6_list)):
                if search_6_list[k][0] in score_a[t]:
                    search_6_list[k][1] = search_6_list[k][1] + 1
        for v in range(len(score_b)):
            for b in range(len(search_6_list)):
                if search_6_list[b][0] in score_b[v]:
                    search_6_list[b][1] = search_6_list[b][1] + 1
    
print("amount of documents: ",len(topic_documents_6))
print(search_6_list)

unique terms:  ['die', 'dead', 'retire']
general terms:  ['old', 'year', 'former', 'hospital', 'cancer']
amount of documents:  3074
[['Term', 'Term Frequency'], ['die', 3617], ['dead', 441], ['retire', 273], ['old', 1302], ['year', 2026], ['former', 515], ['hospital', 865], ['cancer', 2583]]


In [14]:
'''
USES LEMMATIZED SENTENCES
for topic 7: business
company, sale, market, sell, maker, stock, giant, buy, share, drug, yesterday, inc product, marijuana, 
say, trial, social, profit, jury, advertising, inquiry, paper, green, deal, tour, feed, farm, record, pick, bank
'''
search_7 = ["sale", "market", "sell"] #top 3 unique
search_7b = ["company", "buy", "share", "drug", "yesterday"] #top 5 general
print("unique terms: ", search_7)
print("general terms: ", search_7b)
topic_documents_7 = []

w, h = 2, 9
search_7_list = [[0 for x in range(w)] for y in range(h)]
search_7_list[0][0] = "Term"
search_7_list[0][1] = "Term Frequency"
for i in range(len(search_7)):
    search_7_list[i+1][0] = search_7[i]
for i in range(len(search_7b)):
    search_7_list[i+4][0] = search_7b[i]

for i in range(len(data_lemmatized_v2)):
    score_a = []
    score_b = []
    for j in range(len(data_lemmatized_v2[i])):
        if data_lemmatized_v2[i][j] in search_7:
            score_a.append(data_lemmatized_v2[i][j])
        if data_lemmatized_v2[i][j] in search_7b:
            score_b.append(data_lemmatized_v2[i][j])
    if len(score_a) > 0 and len(score_b) > 0:
        topic_documents_7.append(data_lemmatized_v2[i])
        for t in range(len(score_a)): 
            for k in range(len(search_7_list)):
                if search_7_list[k][0] in score_a[t]:
                    search_7_list[k][1] = search_7_list[k][1] + 1
        for v in range(len(score_b)):
            for b in range(len(search_7_list)):
                if search_7_list[b][0] in score_b[v]:
                    search_7_list[b][1] = search_7_list[b][1] + 1
    
print("amount of documents: ",len(topic_documents_7))
print(search_7_list)

unique terms:  ['sale', 'market', 'sell']
general terms:  ['company', 'buy', 'share', 'drug', 'yesterday']
amount of documents:  731
[['Term', 'Term Frequency'], ['sale', 325], ['market', 310], ['sell', 293], ['company', 353], ['buy', 77], ['share', 98], ['drug', 605], ['yesterday', 226]]


In [15]:
'''
USES LEMMATIZED SENTENCES
for topic 10: regulations
rule, food, water, county, exposure, judge, smoke, Chicago, pollution, order, dump, regulation, fish, florida, statement, 
lee, responsible, mary, normal, glass, smoking, politician, iran, predict, warning, emergency, court, tree, oppose, trenton
'''
search_10 = ["rule", "food", "water"] #top 3 unique
search_10b = ["judge", "smoke", "order", "glass", "smoking"] #top 5 general
print("unique terms: ", search_10)
print("general terms: ", search_10b)
topic_documents_10 = []

w, h = 2, 9
search_10_list = [[0 for x in range(w)] for y in range(h)]
search_10_list[0][0] = "Term"
search_10_list[0][1] = "Term Frequency"
for i in range(len(search_10)):
    search_10_list[i+1][0] = search_10[i]
for i in range(len(search_10b)):
    search_10_list[i+4][0] = search_10b[i]

for i in range(len(data_lemmatized_v2)):
    score_a = []
    score_b = []
    for j in range(len(data_lemmatized_v2[i])):
        if data_lemmatized_v2[i][j] in search_10:
            score_a.append(data_lemmatized_v2[i][j])
        if data_lemmatized_v2[i][j] in search_10b:
            score_b.append(data_lemmatized_v2[i][j])
    if len(score_a) > 0 and len(score_b) > 0:
        topic_documents_10.append(data_lemmatized_v2[i])
        for t in range(len(score_a)): 
            for k in range(len(search_10_list)):
                if search_10_list[k][0] in score_a[t]:
                    search_10_list[k][1] = search_10_list[k][1] + 1
        for v in range(len(score_b)):
            for b in range(len(search_10_list)):
                if search_10_list[b][0] in score_b[v]:
                    search_10_list[b][1] = search_10_list[b][1] + 1
    
print("amount of documents: ",len(topic_documents_10))
print(search_10_list)

unique terms:  ['rule', 'food', 'water']
general terms:  ['judge', 'smoke', 'order', 'glass', 'smoking']
amount of documents:  213
[['Term', 'Term Frequency'], ['rule', 165], ['food', 54], ['water', 40], ['judge', 139], ['smoke', 41], ['order', 87], ['glass', 14], ['smoking', 16]]


In [16]:
'''
observations!
'''

'\nobservations!\n'

In [17]:
'''
Topic 2: environmental/health politics
'''
print("INSPECTION OF TOPIC 2: Environmental / health politics")
print("search terms used:")
print("Top 3 specific: ", search_2)
print("Top 5 non-specific: ", search_2b)
print("Subjects found within the documents:")
print("")
print("SUBJECT 1: Asbestos concerns, research & legislation")
T21 = [21,30,1,48]
T22 = [32,45]
T23 = [37,57,90,98]
T24 = [10,40,48,113,758]
for i in T21:
    print("[",i,"]"," ".join(topic_documents_2[i]))
print("")
print("SUBJECT 2: Chemical bans/research (linked to cancer)")
for i in T22:
    print("[",i,"]"," ".join(topic_documents_2[i]))
print("")
print("SUBJECT 3: Pesticides")
for i in T23:
    print("[",i,"]"," ".join(topic_documents_2[i]))
print("")
print("SUBJECT 4: health policies")
for i in T24:
    print("[",i,"]"," ".join(topic_documents_2[i]))
print("")


INSPECTION OF TOPIC 2: Environmental / health politics
search terms used:
Top 3 specific:  ['worker', 'asbestos', 'congress']
Top 5 non-specific:  ['health', 'today', 'federal', 'government', 'official']
Subjects found within the documents:

SUBJECT 1: Asbestos concerns, research & legislation
[ 21 ] asbestos chemical new rule jersey suspect substance list trenton oct pressure federal government new jersey department adopt new air_pollution regulation today include rule design cut pollution asbestos toxic_chemical
[ 30 ] test asbestos exposure new york city public school screen disease because asbestos accord school union official screen program describe official first kind nat
[ 1 ] hear plan risk face asbestos worker late effort pass legislation aid worker suffer asbestos relate illness key congressman say today plan hold hearing decision file reorganization federal
[ 48 ] school asbestos law pose problem lead new jersey may find difficult comply federal asbestos management guideline

In [18]:
'''
Topic 3: resesarch
'''
print("INSPECTION OF TOPIC 3: RESEARCH")
print("search terms used:")
print("Top 3 specific: ", search_3)
print("Top 5 non-specific: ", search_3b)
print("Subjects found within the documents:")
print("")
print("SUBJECT 1: gene therapy/research (cancer linked)")
T31 = [7,31,36,52,72,86,157]
for i in T31:
    print("[",i,"]"," ".join(topic_documents_3[i]))
print("")
print("SUBJECT 2: breast cancer")
T32 = [53,56,57,62,68]
for i in T32:
    print("[",i,"]"," ".join(topic_documents_3[i]))
print("")
print("SUBJECT 3: cancer studies")
T33 = [76,83,101]
for i in T33:
    print("[",i,"]"," ".join(topic_documents_3[i]))
print("")
print("SUBJECT 4: virus research")
T34 = [29, 53, 64]
for i in T34:
    print("[",i,"]"," ".join(topic_documents_3[i]))
print("")
print("SUBJECT 5: aids/hepatitis research")
T35 = [40,67,95,100]
for i in T35:
    print("[",i,"]"," ".join(topic_documents_3[i]))
print("")


INSPECTION OF TOPIC 3: RESEARCH
search terms used:
Top 3 specific:  ['study', 'scientist', 'risk']
Top 5 non-specific:  ['cancer', 'find', 'research', 'aid', 'report']
Subjects found within the documents:

SUBJECT 1: gene therapy/research (cancer linked)
[ 7 ] scientist report first success gene_therapy first time gene_therapy succeed scientist doctor use treatment involve add work gene cell several infant may otherwise die severe immune disorder
[ 31 ] hopeful new cancer treatment company team study gene lead day researcher university california san_francisco receive nobel_prize discovery normal gene cancer go_awry tiny new york receive first food
[ 36 ] genentech share fall concern cancer drug share genentech fell_percent yesterday study raise concern safety experimental cancer drug avastin study find analyst say side_effect bleed
[ 52 ] new genetic risk prostate cancer identify certain gene variation greatly increase man prostate cancer new test may limit use patient expert say
[ 72

In [19]:
'''
Topic 4: health care
'''
print("INSPECTION OF TOPIC 4: HEALTHCARE")
print("search terms used:")
print("Top 3 specific: ", search_4)
print("Top 5 non-specific: ", search_4b)
print("Subjects found within the documents:")
print("")
print("SUBJECT 1: (breast) cancer research")
T41 = [29,35,37,49,36,30,63,103]
for i in T41:
    print("[",i,"]"," ".join(topic_documents_4[i]))
print("")
print("SUBJECT 2: cost of health care")
T42 = [9,12,112,200,196,250]
for i in T42:
    print("[",i,"]"," ".join(topic_documents_4[i]))
print("")
print("SUBJECT 3: marijuana/opioids")
T43 = [22,180,182,415,95]
for i in T43:
    print("[",i,"]"," ".join(topic_documents_4[i]))
print("")
print("SUBJECT 4: drug legislation")
T44 = [134,125]
for i in T44:
    print("[",i,"]"," ".join(topic_documents_4[i]))
print("")
print("SUBJECT 5: undergoing chemotherapy")
T45 = [310,344]
for i in T45:
    print("[",i,"]"," ".join(topic_documents_4[i]))
print("")

INSPECTION OF TOPIC 4: HEALTHCARE
search terms used:
Top 3 specific:  ['doctor', 'patient', 'breast']
Top 5 non-specific:  ['cancer', 'drug', 'medical', 'say', 'woman']
Subjects found within the documents:

SUBJECT 1: (breast) cancer research
[ 29 ] chance removal breast vary tumor wide difference report incidence cancer remain breast woman cancerous breast remove figure report study publish medical journal range per_cent wide_range
[ 35 ] breast cancer link use antibiotic frequent use antibiotic link great risk breast cancer say researcher study american woman find take drug often twice risk disease study uncover
[ 37 ] test breast ultrasound get approval approve powerful ultrasound device today help doctor determine lump woman breast noncancerous woman avoid common surgical test maker device advance
[ 49 ] patent detection method breast tumor may add fire debate patent medical treatment science technology greatly influence patent trend often create whole new patent debate patent medi

In [20]:
'''
Topic 5: AMERICAN POLITICS
'''
print("INSPECTION OF TOPIC 5: AMERICAN POLITICS")
print("search terms used:")
print("Top 3 specific: ", search_5)
print("Top 5 non-specific: ", search_5b)
print("Subjects found within the documents:")
print("")
print("SUBJECT 1: Health campaigns")
T51 = [28,62,91,129,467]
for i in T51:
    print("[",i,"]"," ".join(topic_documents_5[i]))
print("")
print("SUBJECT 2: cancer deaths")
T52 = [361,369,377,492,499,506]
for i in T52:
    print("[",i,"]"," ".join(topic_documents_5[i]))
print("")
print("SUBJECT 3: Polution")
T53 = [108,113]
for i in T53:
    print("[",i,"]"," ".join(topic_documents_5[i]))
print("")
print("SUBJECT 4: Health related bills")
T54 = [5,16,70,106,121,193,220,225,383,48,554,283,220]
for i in T54:
    print("[",i,"]"," ".join(topic_documents_5[i]))
print("")

INSPECTION OF TOPIC 5: AMERICAN POLITICS
search terms used:
Top 3 specific:  ['city', 'jersey', 'campaign']
Top 5 non-specific:  ['new', 'york', 'state', 'school', 'political']
Subjects found within the documents:

SUBJECT 1: Health campaigns
[ 28 ] ford urge flu campaign entire ask congress make vaccine new virus avert fall winter epidemic ford urge flu campaign entire population washington march president ford call today government support campaign vaccinate entire population new influenza virus possible epidemic fall winter
[ 62 ] new tactic emerge struggle smog lead decade progress campaign control smog car truck lose grind enormous increase number vehicle road expert believe new anti pollution strategy may
[ 91 ] grave young man yale lieberman even box evil laud president kennedy southern urge congress tackle new frontier legislation lecture yale_university president first amendment call national smoking campaign
[ 129 ] lazio breast cancer effort assess campaign begin televise se

In [21]:
'''
Topic 6: MORTALITY
'''
print("INSPECTION OF TOPIC 6: MORTALITY")
print("search terms used:")
print("Top 3 specific: ", search_6)
print("Top 5 non-specific: ", search_6b)
print("Subjects found within the documents:")
print("")
print("SUBJECT 1: cancer deaths")
T61 = [48,70,82,242]
for i in T61:
    print("[",i,"]"," ".join(topic_documents_6[i]))
print("")



INSPECTION OF TOPIC 6: MORTALITY
search terms used:
Top 3 specific:  ['die', 'dead', 'retire']
Top 5 non-specific:  ['old', 'year', 'former', 'hospital', 'cancer']
Subjects found within the documents:

SUBJECT 1: cancer deaths
[ 48 ] charle diplomat die envoy moscow paris top expert soviet decade charle envoy moscow paris die kremlin decision intern japanese charle year government influential adviser soviet relation die cancer evening washington medical center year old charle
[ 70 ] bill columnist kansas_city editor kansas_city feb_ap bill associate editor kansas_city star nationally syndicate columnist die today cancer year old
[ 82 ] morris watson newsman die key labor law test case san_francisco feb_ap morris watson early organizer newspaper guild whose test case national labor relation act die saturday cancer year old
[ 242 ] david thompson counsel david thompson serve deputy associate counsel gov robert new jersey die yesterday brain_tumor hospital years_old_lived road



In [22]:
'''
Topic 7: BUSINESS
'''
print("INSPECTION OF TOPIC 7: BUSINESS")
print("search terms used:")
print("Top 3 specific: ", search_7)
print("Top 5 non-specific: ", search_7b)
print("Subjects found within the documents:")
print("")
print("SUBJECT 1: Stocks and market")
T71 = [3,6,14,19,24,112,159,203,296]
for i in T71:
    print("[",i,"]"," ".join(topic_documents_7[i]))
print("")
print("SUBJECT 2: Drug Sales")
T72 = [367,373,403,688,689]
for i in T72:
    print("[",i,"]"," ".join(topic_documents_7[i]))
print("")

INSPECTION OF TOPIC 7: BUSINESS
search terms used:
Top 3 specific:  ['sale', 'market', 'sell']
Top 5 non-specific:  ['company', 'buy', 'share', 'drug', 'yesterday']
Subjects found within the documents:

SUBJECT 1: Stocks and market
[ 3 ] digest economy company market today column genentech sell new share flurry demand see stock rise high closing concern public offer volatile issue come market decade raise
[ 6 ] squeeze hurt health niche profit pressure get patient expensive hospital bed quickly company provide intravenous nutrition drug home roll sale call home company triple year
[ 14 ] market_place biotechnology discovery leave investor wednesday genentech_inc corporation announce significant discovery ever biotechnology yesterday market researcher genentech unit
[ 19 ] company unveil dna device mean cheap british company plan sell gene device size memory stick plug computer deliver result
[ 24 ] company plan sell soviet cigarette lead ease worsen soviet cigarette shortage america la

In [25]:
'''
Topic 10: REGULATIONS
'''
print("INSPECTION OF TOPIC 10: REGULATIONS")
print("search terms used:")
print("Top 3 specific: ", search_10)
print("Top 5 non-specific: ", search_10b)
print("Subjects found within the documents:")
print("")
print("SUBJECT 1: safety")
T101 = [2,11,21,102]
for i in T101:
    print("[",i,"]"," ".join(topic_documents_10[i]))
print("")
print("SUBJECT 2: health regulations")
T102 = [9,11,101]
for i in T102:
    print("[",i,"]"," ".join(topic_documents_10[i]))
print("")
print("SUBJECT 3: food regulations")
T103 = [100,109,194]
for i in T103:
    print("[",i,"]"," ".join(topic_documents_10[i]))
print("")


INSPECTION OF TOPIC 10: REGULATIONS
search terms used:
Top 3 specific:  ['rule', 'food', 'water']
Top 5 non-specific:  ['judge', 'smoke', 'order', 'glass', 'smoking']
Subjects found within the documents:

SUBJECT 1: safety
[ 2 ] city fight order care mentally_ill city lawyer say yesterday giuliani administration would appeal state judge rule continue mental health care must arrange inmate release rikers_island jail lawyer leonard
[ 11 ] act restrict toxic_chemical order tighten regulation pcb use agency may press case sense urgency rule call effective order provide stringent rule control pose serious health threat
[ 21 ] plan seek law food pesticide face court order ban many important farm chemical agency say today would ask congress relax law prohibit cancer food issue
[ 102 ] guilty dump chemical judge find corporation repeatedly violate federal environmental law dump toxic_chemical refinery rule lawsuit_brought several environmental group

SUBJECT 2: health regulations
[ 9 ] amgen w

In [26]:
'''
all found documents
'''

'\nall found documents\n'

In [12]:
'''
Search:
*Topic 2: environmental/health politics
'''
print("amount of documents: ", len(topic_documents_2))
print("search terms unique:", search_2)#xxx documents
print("search terms general: ", search_2b)
for i in range(len(topic_documents_2)):
    print("[",i,"]", " ".join(topic_documents_2[i]))

amount of documents:  760
search terms unique: ['worker', 'asbestos', 'congress']
search terms general:  ['health', 'today', 'federal', 'government', 'official']
[ 0 ] study asbestos country find lethal mineral present everywhere durham scientist country gather midst tobacco country last week grapple urgent public health question risk general public face asbestos mineral
[ 1 ] hear plan risk face asbestos worker late effort pass legislation aid worker suffer asbestos relate illness key congressman say today plan hold hearing decision file reorganization federal
[ 2 ] assess danger health effect asbestos build contentious environmental issue last year expert side issue agree thing asbestos threat fiber escape air whe
[ 3 ] atom worker death inquiry dispute atomic worker union challenge today important aspect federal state investigation death oklahoma laboratory technician charge make plutonium plant work
[ 4 ] justice refuse settle asbestos case today settle conflict insurance liability

[ 90 ] new limit propose administration call today drastic low level worker may expose chemical gasoline pesticide substance know
[ 91 ] quiet signing cancer bill back lazio overwhelming congress passage bill provide health care woman breast cancer plead sign bill support public
[ 92 ] dispute occupational risk keep cyanamid factory close union release health report plant health spa bind brook jan strike production maintenance service worker organic chemical division enter seventh week side predict speedy settlement disagreement
[ 93 ] federal rule cut asbestos workplace issue new rule today cut amount asbestos workplace percent rule administration worker
[ 94 ] yale art volume remove safe building health hazard note target date give volume yale_university art architecture library move new site later week health threat pose asbestos particle fall ceiling
[ 95 ] measure offer congress detection cleanup radon lead member congress today introduce legislation federal program help detect de

[ 176 ] asbestos rule challenge asbestos industry group say today filed_suit challenge emergency action federal limit exposure worker cancer because asbestos asbestos information association base
[ 177 ] put asbestos lincoln tunnel lead federal state agency investigate yesterday bag asbestos find tube lincoln tunnel dump
[ 178 ] tobacco chief say cigarette addictive top executive large american testify congress today believe cigarette addictive would rather child smoke executive sit side
[ 179 ] florida governor lobbying oust federal prosecutor lead gov bob martinez lobby white_house congress replace accuse governor lie jury late series action put prosecutor odd key figure florida
[ 180 ] check datum cancer factory detroit april_ap united automobile worker ask local representative gather health datum check grow suspicion condition automobile plant cancer union president say today
[ 181 ] cigarette job worker tell lead voice concern employee health mostly chicago company announce last w

[ 267 ] join investigation asbestos school federal prosecutor announce yesterday would look asbestos inspection new york city public school city investigator say first time inquiry include company involve
[ 268 ] excerpt presidential panel report mile overall conclusion attitude practice because accident accident worldwide concern raise handle emergency public worker health safety right information utility transition warning section report president commission accident present president carter today
[ 269 ] around_nation arrest parade second crime report show small increase cleveland abortion clinic early burn georgia site compete get carter library senator ask cancer study nuclear ship worker sen thoma ask feed govt investigate health worker may radiation say navy denial problem deserve thorough ind rev boston_globe rept cancer
[ 270 ] toxic_chemical river thwart effort shore dam hold water contaminate formaldehyde worker today rip dam allow water russian river release river
[ 271 ] n

[ 352 ] civil suit texas become criminal matter new york federal grand_jury convene consider possible arise civil litigation exposure asbestos accord lawyer involve civil case
[ 353 ] asbestos pipe replacement urge lead water quality study reveal minute level asbestos area hill water district prompt district official recommend replacement mile asbestos pipe mile system even_though
[ 354 ] coat asbestos fabric term hazardous consumer serious health hazard include possibility lung cancer woman coat make contain asbestos fiber officer international lady garment worker union say
[ 355 ] transcript question answer president carter call die talk phone carter draft woman service concern uganda ease burden health cost aid veteran korean war pay increase congress disagreement drug use transcript carter call continue offer praise restriction concorde invitation minnesota negotiation panama public school commitment trade sec clinton say ban plane violate air service
[ 356 ] asbestos check lag vol

[ 442 ] war cancer stir political backlash war cancer stir political backlash scientist fear vietnam washington may year ago congress president_nixon declare war cancer today see backlash emerge base political scientific aspect effort
[ 443 ] respect man hat squeeze albany last week ton new spending health care rivera head hospital worker union find old time labor boss cast note wear
[ 444 ] paris candidate outline reform make appeal leave pledge ministry criticize right french today announce plan improve life worker age poor quality french life
[ 445 ] health care overhaul hospital drug maker sweep overhaul nation health care system congress would give health care industry customer year
[ 446 ] federal control seek protection nuclear plant federal government assume responsibility protect country safeguard group democratic member congress new york say today speak indian_point
[ 447 ] new drop chief_justice report strike save unit prison official dismiss asbestos worker sue hunt ramsey 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 567 ] congress move revamp rule food safety week congress consider legislation alter rule food safety drastically change scatter several bill propose republican limit federal government authority regulate
[ 568 ] panel say white_house delay effort curb food health claim white_house delay least year attempt curb mislead claim food house panel say report today report reopen dispute congress white_house
[ 569 ] nad unit back job safety bill assail effort change measure labor seek washington nov assert deer better protect work man woman group lead ralph_nader throw support today job health safety bill congress
[ 570 ] head oppose ban birth curb washington ford administration would oppose legislative ban use suspect cancer because hormone secretary tell congress today
[ 571 ] school delay asbestos work nearly public school new jersey fail complete removal asbestos summer unable open next week state education commissioner say today asbestos use insulate pipe
[ 572 ] treat health care middle

[ 654 ] court flood trenton shipyard construction auto part worker handle asbestos flood federal court new jersey damage suit former employer insurance company mining
[ 655 ] urge ban smoking workplace lead public health group ask government today order emergency ban smoke virtually indoor workplace contend tobacco fume worker kill nonsmoker year
[ 656 ] transcript president address congress health care address joint session congress last night plan overhaul nation health care system record new york time speaker president member congress
[ 657 ] around_nation court order wilmington school contact chloroform job oppose pleads_guilty louisiana vote case agency say reject new hampshire aid bid health research group petition bar exposure worker chloroform new evidence carcinogen chicago area com join petition
[ 658 ] around_nation worker protest local treatment woman arrest day siege home saccharin warning_label place today boy store clerk slay woman organize construction worker wife fair 

[ 741 ] house pass spending bill carry big catch congress today move step_closer showdown budget house approve temporary measure would allow government continue spend money dec bill would
[ 742 ] act halt dump lake operation oppose cancer peril ask today immediate shutdown nation big iron mining operation grind discharge asbestos laden waste lake_superior
[ 743 ] chemical say cancer federal researcher urge government examine standard worker exposure cite new evidence chemical main component synthetic rubber may potent cancer researcher national
[ 744 ] cite flaw health panel reject deal tobacco panel nation lead public health expert set advise congress propose tobacco settlement today call agreement unacceptable saying would undermine government ability regulate nicotine
[ 745 ] manville plan question representative asbestos victim claim today plan question proposal would allow company emerge bankruptcy proceeding begin pay
[ 746 ] unfinished business remain legislature albany state se

In [13]:
'''
Search:
*Topic 3: research
'''
print("amount of documents: ", len(topic_documents_3))
print("search terms unique:", search_3)#736 documents
print("search terms general: ", search_3b)
for i in range(len(topic_documents_3)):
    print("[",i,"]", " ".join(topic_documents_3[i]))

amount of documents:  2848
search terms unique: ['study', 'scientist', 'risk']
search terms general:  ['cancer', 'find', 'research', 'aid', 'report']
[ 0 ] tiny patient major goal mouse hospital beth_israel similar elsewhere new approach study human cancer prostate cancer man
[ 1 ] asbestos union manufacturer fund occupational cancer study unusual display cooperation issue labor industry award half dollar yesterday mount_sinai medical center finance attempt find cure because cancer
[ 2 ] clue mixture sexual trait believe find new york scientist find believe may abnormal condition person mixture male female sex trait case structure sex
[ 3 ] datum tie cancer electric_power find false federal investigation conclude scientist lawrence berkeley laboratory berkeley_calif fake consider crucial evidence tie electric_power line cancer disclosure appear strengthen
[ 4 ] tumor fuel steal part host report say group scientist propose venereal tumor sometimes grab produce cell fuel dog infect
[ 5 ]

[ 90 ] health care lag black poor even people cover medicare type care receive overall mortality influence race income accord result large federal study insurance find essential obtain good
[ 91 ] cancer lead simply count number person body may give clue risk dangerous cancer arise certain cell skin tissue
[ 92 ] family size tie brain_tumor sibl likely may develop brain_tumor new study report researcher write dec issue review record swedish brain cancer patient find
[ 93 ] prognosis prostate cancer new man prostate cancer take may lower risk die disease recommend prostate cancer prevention treatment researcher
[ 94 ] review arthritis drug link cancer regulator group arthritis drug make schering_plough company heighten risk cancer youngster
[ 95 ] hepatitis risk east asian new york east asian immigrant new york city carry hepatitis_virus new study find
[ 96 ] cancer drug free brake immune_system show promise hodgkin early stage study merck shrink tumor patient
[ 97 ] mistake find test n

[ 184 ] fitness researcher study effect exercise pregnancy lead obstetrician advise pregnant patient right exercise research recent year find consequence maintain moderate fitness routine pregnant
[ 185 ] chemical find human brain also produce cell animal harold_schmeck hormone new substance evolve fit need complex animal scientist suppose ancient chemical use day life earth
[ 186 ] quotation_day prove people successfully reduce cancer risk quit smoking take smoking director national cancer institute lung cancer report
[ 187 ] study develop way monitor cancer care respond call improve quality cancer care lead cancer organization say today work group develop national system monitor well cancer patient treat
[ 188 ] key human growth scientist know malignant_tumor feed human body must generate new survive grow research team harvard medical school announce last week produce small protein
[ 189 ] animal fat tie colon cancer large study western diet colon cancer ever find red_meat animal fat

[ 277 ] daniel daniel health scientist administrator bureau health service research die yesterday cancer home year old
[ 278 ] note find science fraud overturn federal panel conclusion celebrate investigation allege scientific misconduct federal appeal board yesterday reject find scientist datum decade ago report sign lead
[ 279 ] link cancer birth_defect hint test bacteria genetic trait change relationship dispute link suggest skin study favor possibility hair_dye may cancer birth_defect raise result laboratory test bacteria conduct university california biochemist
[ 280 ] drug find supplement breast cancer surgery drug find supplement breast cancer surgery drug combination treatment surgery breast cancer dramatically reduce recurrence cancer patient face high risk suffer relapse accord report week italian researcher
[ 281 ] headline report study cancer wife smoker washington june_upi tobacco institute report today may critical error japanese study report wife cigarette
[ 282 ] study 

[ 368 ] scientist report find way researcher report today laboratory experiment animal discover novel potentially powerful way make tumor single injection type protein protein
[ 369 ] cancer risk hormone may linger scientific evidence show woman avoid breast cancer risk take drug short time researcher say today also show risk disappear soon drug
[ 370 ] fact kill disease great advance medical science cancer continue exact heavy toll american detail major killer list yesterday study release national health
[ 371 ] researcher find snuff increase risk oral cancer grow habit dip snuff favor people alternative smoking cigarette significantly increase risk mouth cancer accord government study dip snuff put inside mouth lip
[ 372 ] man get baboon marrow risky aid treatment radical cross specie experiment risk implication treat many disease scientist last night inject bone_marrow remove baboon year old man aid hope
[ 373 ] discovery tumor may point cancer vaccine scientist long seek design can

...

[ 1670 ] scientist whose discovery help combat cancer honor scientist make discovery date identify genetic basis cancer improve diagnosis treatment disease receive year albert_lasker award recipient find
[ 1671 ] study hint grape red_wine may inhibit cancer grape red_wine already herald potential prevent heart_disease may also harbor potent cancer accord study publish today journal science researcher university illinois show
[ 1672 ] obesity add danger man prostate cancer man prostate cancer obesity link increased_risk cancer aggressive likely recur surgery study find connection obesity may help explain racial difference prostate cance
[ 1673 ] scientist commemorate dna discuss function reverse twist dna twist wrong way may play role regulation gen cancer process scientist say today dna acid active substance gene live thing dna find
[ 1674 ] study question value prostate cancer surgery new study show prostate cancer surgery often leave man appear man early stage disease account case ma

[ 1937 ] toms_river cancer report release state health official last week say toms_river resident may expose carcinogen town water_supply chemical plant report part study
[ 1938 ] new datum pill find breast cancer possible risk lead expert meet today consider surprising new evidence use may increase_risk breast cancer
[ 1939 ] anemia_drug may impair cancer treatment drug widely prescribe anemia cancer patient believe help radiation work good prolong_life may even cancer treatment patient study europe find study publish today
[ 1940 ] new test find potential peril research scientist state department say last week new broad base test industrial discharge river atlantic_ocean indicate potential damage fish
[ 1941 ] study link virus class journal science publish attention grab study last fall link syndrome recently discover many expert remain skeptical especially study find association
[ 1942 ] anticancer substance reveal dark side potentially important anticancer substance lead double lif

[ 2263 ] link find cancer long_island study woman long_island find link breast cancer live high extend long trail research fail connect breast cancer island environmental report
[ 2264 ] report suggest link gene usher politically explosive study origin new perhaps rigorous phase researcher report link male small region human chromosome
[ 2265 ] study report small risk breast_implant large study far disease find implant pose health risk small long_awaited study however already use side bitter controversy
[ 2266 ] say result cancer drug trial pharmaceutical say yesterday study find drug fail prolong_life use chemotherapy common form lung cancer company stock fell_percent company seek
[ 2267 ] menopause hormone may good woman hop combat alternative soy supplement recent study hold disappointment last week clinical_trial find soy work better
[ 2268 ] citizen scientist obligation stand standard april appear symposium student teacher sponsor illinois math science academy remarkably successfu

[ 2522 ] job cut hurt life science rally resemble armed camp public access candidate limit federal policy recession may severely hamper research work generation biomedical scientist group expert warn
[ 2523 ] smoke cancer cigarette concern really know lead early year warning_label worry cigarette could cancer independent researcher begin publish major study health
[ 2524 ] smoke cancer news really news anybody headline proclaim little week ago smoking because lung cancer discovery news story report team scientist show chemical cigar
[ 2525 ] cancer study prompt surge desire read yesterday morning new study researcher cornell medical center show could provide early_detection lung cancer smoker former smoker richard call cornell find could
[ 2526 ] major study dispel fear possible harm term health danger far popular birth_control method man major new study find year later man healthy researcher report dalla
[ 2527 ] gain find cancer victim survival_rate year survival_rate patient common 

[ 2612 ] job major concern nuclear shipyard town satisfy safety measure concern study find feb heart town alice waterfront bar restaurant main gate shipyard jack alice husband
[ 2613 ] cancer transfusion study almost patient rectal colon cancer find recurrence high receive transfusion whole blood rather red surgery researcher university
[ 2614 ] biopsy study tissue biopsy involve removal small piece suspect tissue diagnosis evaluation naked eye microscope cancer possibility pathologist compare tissue size color shape
[ 2615 ] deliberate kill lead death lead suicide homicide lead because premature death american year accident cancer heart_disease accord new federal study
[ 2616 ] composer dead write song classical work let unique also admire make other laugh return study eastman home composer whose work serious popular music set contemporary die lung cancer yesterday morning north florida regional hospital year old
[ 2617 ] califano order step halt sale many sleep drug may joseph_califa

In [14]:
'''
Search:
*Topic 4: health care

'''
print("amount of documents: ", len(topic_documents_4))
print("search terms unique:", search_4)#xxx documents
print("search terms general: ", search_4b)
for i in range(len(topic_documents_4)):
    print("[",i,"]", " ".join(topic_documents_4[i]))

amount of documents:  3370
search terms unique: ['doctor', 'patient', 'breast']
search terms general:  ['cancer', 'drug', 'medical', 'say', 'woman']
[ 0 ] body discovery investigate authority find body believe woman doctor claim help commit_suicide body community college plan use take county medical
[ 1 ] tiny patient major goal mouse hospital beth_israel similar elsewhere new approach study human cancer prostate cancer man
[ 2 ] grim diagnosis still goal carole doctor tell june cancer year live take way say always take doctor advice inform opinion consultant may may
[ 3 ] breast cancer treatment reader react article thursday treatment prompt wide_range response anger sadness relief
[ 4 ] look mom date whole bunch doctor really year novel blend speed medical earn sky high rating long_term relationship know
[ 5 ] break rank lab offer test assess risk breast cancer small world geneticist study human disease researcher tend know last year discovery many percent jewish woman carry form gen

[ 97 ] chronicle question bradley wife senator bill_bradley theme luncheon celebrate life honor next week breast cancer survivor whose disease diagnose year
[ 98 ] business brief regulatory legal news delay rule drug breast cancer appear postpone expect decision revoke approval drug avastin treatment breast cancer genentech manufacturer drug say friday
[ 99 ] new jersey_brief bell say plan replace goldstein ray patient seek byrne utility rate clash water purity clash design protect water quality jersey water resource aide brown hearing link commercial development give authority rev plan
[ 100 ] headline diagnosis state jury long_island award queen man whose doctor illness cancer tell month live
[ 101 ] medicare patient tell poor doctor care win right learn find federal judge order medicare official disclose finding whenever investigate consumer complaint doctor hospital provide poor quality care commit medical error injure patient ruling
[ 102 ] terminal cancer patient live find propon

[ 190 ] kevorkian partner face cancer janet good take sleep bit ever doctor tell terminal pancreatic cancer last year tuesday get early could drive county halfway michigan charge
[ 191 ] aide navy transport washington may deputy public affair officer navy military sea transportation service die cancer doctor years_old_lived silver spring
[ 192 ] drug fight breast cancer approval yesterday recommend approval drug tamoxifen reduce woman breast cancer drug already market intend
[ 193 ] guideline urge mammogram woman month controversy mammogram federal health official today issue new guideline breast cancer screen strongly recommend test low age woman begin receive
[ 194 ] question choice foundation drug program novarti say choice max foundation run ambitious program give largely result foundation success establish bone_marrow registry leukemia patient registry promote
[ 195 ] robert cole doctor crisis cole late dozen young man woman kenneth clark direction academic black already veteran m

[ 283 ] medicare credit closing gap black woman die breast cancer high rate white woman comparable tumor gap disappear woman old enough qualify medicare accord study release yesterday study author suggest find
[ 284 ] shah doctor report make progress pahlevi whose condition describe early month critical report doctor slowly improve egyptian doctor medical team remark
[ 285 ] china ill wealthy look abroad medical treatment hospital new generation medical tourism company lure chinese patient away health care system
[ 286 ] little know medical field treat pain disability pain sufferer accident victim patient survive stroke cancer debilitate disease well countless tennis player age insist remain active old age finding
[ 287 ] datum mount avoid breast cancer chemotherapy evidence genetic test help predict breast cancer recur could aid doctor patient decide use chemotherapy
[ 288 ] gov_grasso excellent health follow radiation doctor say hartford july_upi recuperate radiation treatment order 

[ 378 ] demand surge prostate test joseph urologist minn recently poll practice doctor state ask use widely promote screen test prostate cancer percent say routine
[ 379 ] cancer progress statistic telling_truth small grow band distinguished analyst challenge government official lead cancer scientist great advance make cure cancer patient analyst suggest highly
[ 380 ] debate radiation breast cancer radiation treatment prescribe breast cancer patient include woman would tell year ago could skip add therapy mostly affect woman treat
[ 381 ] patient dilemma clinical_trial recruit sick people study experimental drug company obligation patient people sign accept risk hop benefit receive free medical care drug
[ 382 ] doctor link gene colon cancer month identify gene thought may set colon cancer researcher say gene true disease gene sit right next_door chromosome
[ 383 ] federal official cite deficiency harvard hospital federal health official issue report yesterday cite wide_variety seriou

...

[ 2429 ] virus clue find breast cancer mouse carcinogen call link human disease washington march new evidence suggest virus factor because human breast cancer report international team scientist
[ 2430 ] vital drug run low base ingredient many kitchen supplier simple critical medication sodium solution run leave doctor chemotherapy postpone surgery
[ 2431 ] business health new test cancer lead imagine patient symptom cancer whose laboratory test produce inconclusive result doctor order treatment tell patient home try worry
[ 2432 ] waste cancer drug cost year study say many drug maker sell vial mean patient pay medicine never give researcher say
[ 2433 ] levine dilemma morning levine duck exam room medical assistant give patient office sit computer take note possible case sit exam room young woman
[ 2434 ] researcher find first drug know prevent breast cancer first time drug show prevent breast cancer woman high risk disease federal health official say today group woman take drug tamox

[ 2522 ] new technique may point coronary risk cheap set test develop lead medical center around_country promise predict heart_disease stroke pinpoint patient really need aggressive therapy far traditional risk
[ 2523 ] award family sue sign surge conflict medical cost california jury award family woman whose health refuse pay costly experimental procedure treat breast cancer
[ 2524 ] risk report chill use cheap eye drug robert specialist brooklyn bronx say want save patient medicare money treat eye disease drug cost dose rather cost dose recent incident
[ 25252680 ] reportedly delay pay hospital doctor hospital official doctor say yesterday insurer manage health care company new york state delay payment medical bill force hospital borrow money pay
[ 2681 ] regimen tie drop breast cancer thorough new study california woman find moderate regular reduce woman breast cancer much percent benefit see great
[ 2682 ] personal health typical question request receive answer federally sponsor ne

[ 2775 ] cancer patient workshop write healing could argue crucial first step make emotional day breast cancer diagnose august continue creative writing class sarah_lawrence college
[ 2776 ] breast good baby sometimes mom ne help gay new york pediatrician enlist nursing mother supply breast milk baby partner adopt baby month old thrive donate milk part town year old bronx woman
[ 2777 ] breast biopsy leave room doubt breast biopsy good tell difference healthy tissue cancer less reliable identify subtle abnormality new
[ 2778 ] breast cancer death rate find increase lead rate death breast cancer increase high northern state federal health official say
[ 2779 ] prescribe drug specialize art precise science give identical drug patient suffer disease often experience radically different outcome doctor long
[ 2780 ] cancer institute propose limit breast ray washington aug new set guideline use ray technique detection breast cancer national cancer institute recommend week ray use routinely w

[ 3026 ] roosevelt doctor say last illness prevent president perform duty washington april president total command faculty last illness rumor contrary accord medical report publish tomorrow th_anniversary death
[ 3027 ] doctor get rebuke first public appearance force resign breast cancer study coordinate decade bernard fisher receive stand week peer national cancer meeting
[ 3028 ] butler begin treatment outfielder brett_butler begin radiation treatment throat cancer yesterday treatment continue week start clinic atlanta family friend say doctor remove cancerous
[ 3029 ] new york discipline doctor role suicide new york state health department yesterday say would bring charge professional misconduct rochester doctor admit role cancer patient suicide last month grand_jury rochester decide
[ 3030 ] often watch prostate cancer enough eddie contractor find prostate cancer age primary care doctor urologist urge prostate remove hearing wait program tal
[ 3031 ] mastectomy unexpected blow new 

[ 3288 ] cancer patient social woe seminar hear victim plead honesty adjustment st_petersburg beach_fla_march kelly year old father former newspaper editor iowa also cancer patient may die disease
[ 3289 ] health personal health lead early year national cancer institute cancel plan year study woman determine risk breast cancer could greatly reduce woman cut half amount fat eat
[ 3290 ] health official reject plan report medical mistake federal health official say time embrace recent call new federal law require hospital report mistake serious injury death patient proposal
[ 3291 ] learn live die cancer patient get together make today count talk matter friend family find taboo live die kelly sit late night recently office home iowa read mail mostly consist letter cancer patient relative window look town dark
[ 3292 ] patient high bill insurer fight generic big cover part patient payment drug insurer say practice raise overall cost care
[ 3293 ] chronicle walk raise_money breast cancer w

In [16]:
'''
Search:
*Topic 5: american politics
'''
print("amount of documents: ", len(topic_documents_5)) 
print("search terms unique: ", search_5)#736 documents
print("search terms general: ", search_5b)
for i in range(len(topic_documents_5)):
    print("[",i,"]", " ".join(topic_documents_5[i]))

amount of documents:  2184
search terms unique:  ['city', 'jersey', 'campaign']
search terms general:  ['new', 'york', 'state', 'school', 'political']
[ 0 ] west orange dec lawyer new jersey new york die yesterday beth_israel hospital newark cancer years_old_lived old road
[ 1 ] fight mayor propose smoke ban basis health economic smoke opponent face restaurant bar owner today first hear mayor proposal ban smoke public place new york city would strict
[ 2 ] koch pick head woman unit kay vice_president cbs sport name chairman new york city commission status woman mayor_koch yesterday
[ 3 ] dioxin new jersey late environmental headache newark many urban ill recently add environmental byproduct manufacture dioxin deadly chemical know man read part
[ 4 ] tower proposal pit medicine development fear landscape new york city never constant long power political ideal progress still debate civic development seldom touch life death future scientific progress
[ 5 ] bill allow stem_cell work clear 

[ 94 ] new cash cow hospital cancer center new york city patient enjoy live classical_music concert newport beach_calif george brennan dermatologist sell skin care cream
[ 95 ] director city ballet school dead school american ballet official school new york city ballet die yesterday cancer year old
[ 96 ] labor speak many voice city mark labor day people build new york people make city put house apartment house office building build subway system lie concrete
[ 97 ] new york area receive toxin taint michigan bay pound bay michigan sample find contain residue chemical dioxin annually ship market new york northern city federal food
[ 98 ] habitat west street jimmy walker live well maybe part house west street jimmy new york city mayor live heavy iron gate unusual block mixed
[ 99 ] poor rock close clinic low_income woman new york city must scramble find routine care clinic use receive close scale lack money victim city economic pro
[ 100 ] author question real imagine impose self impose 

[ 190 ] teenager leave group home challenge place remain month new york city bold experiment move troubled teenager group home foster care many involved worry system stretch thin
[ 191 ] first ounce marijuana leave new jersey open state first medical marijuana montclair come year passage law make possible affair
[ 192 ] plant site assail federal agency criticize would plan meet agency standard plant arouse concern debate reactor accident senior official say swear statement place indian_point new york city emergency response plan serious
[ 193 ] city_council vote strict legislation city_council pass nation tough restriction outdoor cigarette advertising yesterday vote overwhelmingly approve bill would ban advertise new york city bar foot school
[ 194 ] pride new jersey stadium design comfort fan giants_stadium build comfort fan pride new jersey east_rutherford oct water fountain foot rest room patrol special handicap tip policy also politician perhaps
[ 195 ] snow denver area brace new 

[ 283 ] quantity equal quality evaluate legislature appear new jersey weekly recently article concern activity session new jersey legislature article consist largely list program enact begin last month
[ 284 ] grant scher aid medical study hoboken medical research program design bring together doctor research scientist engineer institute new jersey college newark steven
[ 285 ] group portrait new york old old people age city fastest_growing group almost invisible new yorker share story love pain change
[ 286 ] decade pension sweetener year state pass bill order new york city improve pension city worker often city objection pension sweetener approve state legislature
[ 287 ] see thing read palm computer write psychic colonial america law still new york pennsylvania connecticut new jersey age give way new age sort
[ 288 ] inquiry fault chemical city witch brew chelsea blast federal investigator say commercial tenant city share responsibility explosion rip apart chelsea building last spri

[ 374 ] cure city noise much way begin appear ear subway rider replace steel wheel roll stone show ear new yorker small
[ 375 ] board demand asbestos test school queen contract community school board threaten yesterday close city board_education conduct test next thursday dangerous amount asbestos
[ 376 ] royalty long housewife real housewife new york city guide offer tip hang man reader respond recently separate author
[ 377 ] notre_dame quarterback new jersey neighbor division quarterback live golf shoot bergen_county chris likely stay close home college mother fight breast cancer senior season
[ 378 ] overflow day science meaning world science festival new york city organizer may create new cultural institution
[ 379 ] open statement hear william trial senator harrison william new jersey portray yesterday public official scheme make illegally trust man self interested
[ 380 ] doctor kill parent find guilty new jersey judge today acquit doctor charge elderly parent month ago psychiat

...

[ 1301 ] medical society committee ask curb cancer because pollution atlantic_city may committee medical society new jersey urge action today reduce air_pollution state potential because cancer
[ 1302 ] rise cancer mortality find man jersey survey cancer death new jersey year show death rate increase man decrease woman state mortality_rate become closely national average survey
[ 1303 ] new jersey seek strict limit chemical drinking_water panel scientist recommend nation stringent limit class chemical find drinking_water pose_threat public health
[ 1304 ] carry college tradition sharon hold month old daughter shoulder bounce slightly smile girl far perch press box lion stadium college new jersey woman lacrosse team
[ 1305 ] gotti ready trial face anonymous jury chilly spring morning last year playing card queen social club john_gotti arrest charge squad federal agent new york city detective arrest march mark
[ 1306 ] life suspect new jersey murder lead man say robert clark apparently s

[ 1561 ] ultraviolet face revival fight drug resistant strain tuberculosis spread big city researcher new york begin city test later year year old technology kill bacteria ultraviolet
[ 1562 ] times_square plan hold meter still run state city official prepare postpone indefinitely construction times_square center office tower interest continue grow loan public eventually
[ 1563 ] doctor suspend hepatitis dispute practice pennsylvania agreement board question drug role state board raise estimate number hepatitis case attribute peter marco suspend physician resume treatment new jersey patient
[ 1564 ] soul open heart needy new york firmly believe charitable city world nowhere eager help know help want nowhere army devoted worker nowhere abundance mean ready
[ 1565 ] jersey willing give addict clean needle new jersey health official want offer drug addict clean needle dirty part program combat spread aid needle exchange program already use enable health official
[ 1566 ] prisoner witness 

[ 1807 ] may day replace ocean year boat haul load sludge residue sewage treatment plant spot mile east new jersey shore town sea bright dump cargo sludge sink foot sea wat
[ 1808 ] builder urge forge plan times_square city state official today formally ask proposal transform times_square sex fast food shop rat movie house sweep gleam office building restaurant legitimate theater real
[ 1809 ] nbc president lobby city block dredge bill corporation battle federally require billion_dollar cleanup hudson_river upstate new york extend city hall visit president company nbc television network
[ 1810 ] year new jersey resident probably able know live specific abandon dump downwind industrial increase risk contract cancer disease complicate
[ 1811 ] new jersey journal morris county medical society appeal halt uranium prospect standard oil company ohio hill country jefferson township study society ask moratorium
[ 1812 ] resident see health risk new generate plant day hold public hearing propos

[ 1899 ] radium plant queen agree clean site lead radium supply company new york city cite radiation safety violation promise state soon empty plant queen
[ 1900 ] new york manhattan cancer test urge city state increase number van equipment perform mammogram alan say yesterday uninsured woman access free screen breast cancer key step
[ 1901 ] lavish party connor tribute unprecedented year archbishop new york powerful figure city america celebrate cardinal_john joseph connor lavish fund_raising
[ 1902 ] italian american come number way assess quiet yet spectacular rise italian american today good perspective floor window governor new york city office world trade_center
[ 1903 ] diary urban priest new york city kid newly think contemplate assignment raymond section bronx son company executive raise westchester
[ 1904 ] holder tough jersey job daniel joseph man news small town mayor thank friend trenton may ceremony outer office shortly noon today governor_byrne say present new commission

[ 2155 ] peter coach ail peter roy miller hospitalize colon cancer unlikely return season school official say yesterday jersey city cancer detect year old miller
[ 2156 ] new york character sound image start explore city street foot man fixture subway_station would get nearby stop walk high_school travel widely return
[ 2157 ] food irradiation fight center jersey heavily shield industrial building state ton burst radiation rid fungus germ irradiation plant build state
[ 2158 ] new jersey newark miss boy find school bus driver driver aide fire leave year old retarded boy bus hour police_officer find child still bus whi
[ 2159 ] concerned cancer state environmental aide assert government program ban harmful emission trenton sept federal official tell state legislative commission today presence cancer causing_agent air section new jersey great concern something
[ 2160 ] smoke restriction prove popular hard enforce curb public smoking prove popular hard enforce rise nonsmoker part evidence

In [17]:
'''
Search:
*Topic 6: mortality
'''
print("amount of documents: ", len(topic_documents_6))
print("search terms unique: ", search_6)#xxx documents
print("search terms general: ", search_6b)
for i in range(len(topic_documents_6)):
    print("[",i,"]", " ".join(topic_documents_6[i]))

amount of documents:  3074
search terms unique:  ['die', 'dead', 'retire']
search terms general:  ['old', 'year', 'former', 'hospital', 'cancer']
[ 0 ] rule dead jame teamster union leader vanish year ago widely believe victim execution declare dead accord michigan law rule
[ 1 ] public grieve pop_singer startling iran iranian authority catch guard weekend people join gather mourn remember pop_sing die cancer age
[ 2 ] west orange dec lawyer new jersey new york die yesterday beth_israel hospital newark cancer years_old_lived old road
[ 3 ] charle abram worldwide housing expert die lawyer author shape outlook many country foe discrimination head commission charl urban planner whose view major effect housing abroad die cancer home west_th_street yesterday year old
[ 4 ] king king lawyer former president real_estate board die cancer yesterday nassau hospital mineola years_old_lived street floral_park queen
[ 5 ] actress die star love lucy show program longtime star love lucy television se

[ 103 ] mrs henry elizabeth wild past president north_shore junior league active charity long_island die friday cancer roosevelt hospital years_old_lived roslyn
[ 104 ] genetic testing falls_short public embrace ever perfect person test colon cancer gene year old maryland woman name colon cancer family mother die year ago age test show
[ 105 ] katherine dead cancer choreographer katherine choreographer dancer actress know dance solo die cancer tuesday new york hospital year old
[ 106 ] bernard health agency administrator leave post die accept post challenge bernard former health service administrator new york appoint director school university die cancer today woman college hospital year old
[ 107 ] surgeon attend surgeon cancer also chief colon service cancer center die thursday suffer fall
[ 108 ] charle denver writer french politic die prof charl university denver graduate school international study die sunday cancer denver year old professor nationally recognize authority french no

[ 207 ] pakistan little master die aged former pakistan captain little master year give india die age world govern body say thursday
[ 208 ] quotation_day show get government take system turn quietly die breast cancer patient learn aid group
[ 209 ] tobacco middle name law_firm thrive early november year die lung cancer rose_cipollone join ross peter peter number other lengthy list loser survivor take
[ 210 ] year plan quit cbs early bob reporter nearly decade host face nation year say tuesday night would retire next january
[ 211 ] critic take aim former tobacco farm remark democratic convention sister nancy die lung cancer shadow question former ownership tobacco farm home state tennessee
[ 212 ] barbara lilly die find bronxville service unit barbara lilly formerly bronxville wife william chairman board eastern pipeline company die cancer sunday houston years_old_lived houston
[ 213 ] whitney jazz critic dead jazz critic new yorker magazine year vivid highly write encourage reader he

[ 315 ] mary walsh publish official mary walsh former vice_president day company book publisher wife richard walsh former president company die yesterday flower fifth_avenue hospital follow several year treatment canc
[ 316 ] sydney top designer woman fashion dead twice award color family enjoy golf sydney lead woman manufacturer year die last night boca_raton fla cancer bone_marrow year old
[ 317 ] unusual molecule could key cancer patient weight_loss extreme weight_loss know waste long recognize hallmark cancer puzzle researcher decad patient literally waste away acquire almost look actually die waste rather
[ 318 ] poor textile city film joy die cancer last week leave child week job star turn superman role take fun pride pay
[ 319 ] old age catch oil patch robert start call month ago sell old nearly dry oil well mother inherit father die younger_brother jay list jay price oil low know
[ 320 ] mrs mrs carry business many year die cancer yesterday parkway hospital year old widow natha

[ 424 ] paul martin editor news world report washington may upi paul martin retire late last year political editor news world report die today cancer arlington county hospital year old
[ 425 ] lawyer celebrity dead practice soon exam ellen lawyer specialize motion_picture case celebrity client friend die last night cancer center
[ 426 ] die board officer real_estate dealer die cancer sunday jamaica hospital queen years_old_lived tower hotel brooklyn
[ 427 ] notes_people jump thwart actor studio award old story reagan dancer hold news_conference senator little bit cancer report cancer treat oral medication injection year old retire
[ 428 ] leave late bishop diocese paterson paterson july_upi bishop lawrence casey paterson leave estate estimate value accord file passaic_county bishop die cancer june
[ 429 ] charl professor post charl professor american history post college president city club die saturday new york hospital follow treatment cancer queen
[ 430 ] shaw secretary bronx school

...

[ 1987 ] struggle inspire gift lead struggle hold family together wife die cancer year ago inspire number contribution new york fund
[ 1988 ] woman bury husband feud wife sister begin lie die cancer persist court sister dorothy insist brother die wish wife christine
[ 1989 ] joan dead aid channel fund_raiser gather chronicle donor joan principal channel public television production die cancer sunday home year old
[ 1990 ] early_detection american would agree good way avoid die cancer apart get first place catch tumor early possible think half_century cancer
[ 1991 ] alfr die geneticist medal april alfr henry major figure american genetic many year former professor genetic california die cancer hospital last night live
[ 1992 ] coach iowa state former national year lead university final former head coach state university die last night cancer year old
[ 1993 ] mrs author charlotte singer author robert die cancer yesterday cancer research years_old_lived west_th_street
[ 1994 ] jame pain

[ 2272 ] murray dead novelist critic book editor murray novelist poet critic book editor national observer die cancer thursday home age
[ 2273 ] therapy assist help end young woman depression aid fund miss life long_island year old father die leave young sister mother support ever miss suffer depression subsequent event deepen
[ 2274 ] billy comedian die film withdraw irene billy comedian actor die yesterday undergone treatment lung cancer age whose first job show business dancer perform broadway television
[ 2275 ] robert art editor die also new direction officer robert director editor art book publish company editor director vice_president new direction publishing corporation die cancer sleep yesterday
[ 2276 ] professor law former counsel die berkeley_calif dec former general_counsel agency international development university california hall school law die cancer yesterday
[ 2277 ] william kaiser chief engineer william kaiser chief engineer voyage retirement die cancer sunday rise 

[ 2537 ] science_watch retire cancer need cure black cool skin cancer diagnosis west african missionary later find simple stain because dead british medical journal report
[ 2538 ] jame adam industrialist backer medical research die palm_beach jame adam industrialist active supporter humanitarian die live last year year old longtime resident new york city
[ 2539 ] cancer test independent laboratory pap_smear perform newport hospital woman test negative time year die cervical cancer last spring pap_smear far
[ 2540 ] son choose run mother seat nassau_county announce today choose craig johnson lawyer run seat county legislature leave vacant mother barbara johnson die earlier_month cancer republican expect
[ 2541 ] justice franci serve state state justice francis die today cancer irv hospital year old justice republican select year term bench year
[ 2542 ] letter burn bridge reward retirement spiritual letter turn point live letter hang gentle shape smokeless also star letter woman real s

[ 2807 ] die record artist relation friend famous musician director artist relation angel record close_friend musician maria price carlo maria die yesterday beth_israel hospital year
[ 2808 ] union_carbide study relate chemical employee death analyze charleston_va announce today would survey health history employee determine die certain kind cancer rate time normal
[ 2809 ] russell russell administrative medical officer center retirement die cancer hospital sunday year old living
[ 2810 ] dead cancer dead cancer mourn quiet become third secretary general hold post long person die yesterday center year old
[ 2811 ] tie amateur title end competitive career tie amateur owner hunter championship last fall national horse show die cancer sunday greenwich hospital connecticut year old live
[ 2812 ] memoir suggest priest urge die pancreatic cancer last november joseph archbishop chicago write memoir last year spiritual reflection cop disease accusation sexual_abuse
[ 2813 ] harriet harriet cit

[ 3073 ] marvin marvin marvin psychologist die yesterday cancer years_old_liv terrace


In [18]:
'''
*Topic 7: business
'''
print("amount of documents: ", len(topic_documents_7))
print("search terms unique: ", search_7)#426 documents
print("search terms general: ", search_7b)
for i in range(len(topic_documents_7)):
    print("[",i,"]", " ".join(topic_documents_7[i]))

amount of documents:  731
search terms unique:  ['sale', 'market', 'sell']
search terms general:  ['company', 'buy', 'share', 'drug', 'yesterday']
[ 0 ] chinese agency criticism sale fake good day government regulator company show sign move quickly
[ 1 ] fall volume remain heavy generally clinton kind market yesterday large stock continue small issue gain close pierce first time mid november
[ 2 ] clarke marketing official time clarke founder president sell area market inc market subsidiary time inc retire die cancer yesterday stamford_conn hospital years_old_lived sta
[ 3 ] digest economy company market today column genentech sell new share flurry demand see stock rise high closing concern public offer volatile issue come market decade raise
[ 4 ] abbott quit long fight official say give year fight restore cyclamate sweetener market continue produce country news abbott yesterday follow dec
[ 5 ] panel reject expand use prostate drug drug support change drug label would second class ca

[ 99 ] upjohn cancer drug alliance upjohn company say yesterday agree alliance inc company develop make market cancer treatment drug upjohn agreed_pay cash new
[ 100 ] suit continue foundation month dispute member board john catherine macarthur foundation agree unanimously recently sell insurance company own harmony president
[ 101 ] privacy violation medical record find widespread denver base concern recently indict steal medical record sell information contain life insurance company represent tip nationwide accord head recently form
[ 102 ] employer could opt insurance market raise other cost official worry company healthy employee self insure leave commercial insurer number people drive premium
[ 103 ] ruling may hurt amgen right drug lead number lawyer specialize biotechnology warn court decision last week patent right new drug worth could devastate amgen_inc market pioneer
[ 104 ] brief coca_cola say sell upstate new york bottle property include bottle facility albany distribution

[ 199 ] notes_people carter family sell mississippi paper gold card olympic buy man surgery longer dump kidnap victim back parole guilty man musician retire barbara jane georgia pardon parole board jame morris chronicle article say barbara jane approve parole last may gary steven convict kidnapping
[ 200 ] miracle drug system fault washington market year ago seem true miracle drug drug apparently free risk would blood lead death
[ 201 ] cancer drug face fund cut bush plan soon propose significant cut medicare payment cancer drug base new datum suggest government pay far market price medicine administration official say today
[ 202 ] aid drug patent challenge suit lawsuit announce today challenge company patent aid treatment drug azt seek company could market drug cheap price federal court
[ 203 ] week end market loss technology share especially hard hit major index bad week may standard_poor stock index fell_percent
[ 204 ] group copy bid speed sale copy manufacture abortion right grou

[ 300 ] abuse widespread medical sale care home federal official expert say company abuse medicare home health care program sell elderly people worth medical equipment need court record show supplier
[ 301 ] diary drug medicine market broad market slide drug company share climb recommend approval mean breast cancer zeneca_group make drug barr
[ 302 ] rise concern profit send retreat day perch yesterday renew concern corporate earn major market measure also slip worry first set coca
[ 303 ] generic group put eastern european operation sale source generic firm put central eastern european operation sale private equity owner seek step step exit maker drug cancer heart lung disease people close matter say
[ 304 ] say purchase forward laboratory corporation america hold medical laboratory company say federal antitrust review period expire purchase system inc agreed_buy last month increase sale cancer
[ 305 ] company end use apple chemical lead manufacturer apple agent find cancer end chemic

[ 398 ] end bond unit merger hint deny reduce company nation_largest third market dealer long new york stock_exchange cut operation strong report wall_street may move merger
[ 399 ] cyanamid net decline company today say fell_percent year early help sale subsidiary month end sept cent
[ 400 ] sale cancer drug help roche post increase roche_hold report percent surge first quarter revenue drive cancer treatment avastin bird flu_vaccine
[ 401 ] ford urge ask company push cigarette medical effect national cancer advisory_board propose today president ford make personal public appeal market low nicotine cigarette
[ 402 ] could corporate gate last month michael chief_executive company manufacture medical device believe company market price would rise could get investor pay attention
[ 403 ] clear sale receive approval sell stroke make company first manufacturer clear sell version device
[ 404 ] waksal father name suit yesterday sue jack waksal father samuel_waksal founder contend sell worth 

[ 497 ] world switzerland poor sale drug maker roche_hold report first quarter revenue percent less market expectation pharmaceutical company rocky lately demand drug obesity acne
[ 498 ] imclone drug fail pancreatic cancer study say yesterday cancer drug erbitux fail help pancreatic cancer patient live_longer study setback company plan expand market product research testing erbitux
[ 499 ] inquiry britain uncover drug trial trial new drug london nearly kill man reveal testing system seem good bring drug market protect human safety
[ 500 ] drug shows_promise advanced prostate cancer use chemotherapy drug already market post chemotherapy use could open new front competition johnson_johnson
[ 501 ] charter consolidate may sell metal concern stake charter consolidate say yesterday consider sell percent stake johnson metal process concern spokeswoman charter say london base mining
[ 502 ] fish tale fall live morning fish market light stadium night game light mount high create day night say

[ 597 ] large drug concern post strong quarterly gain big drug maker report strong yesterday gain eclipse news ready enter lucrative crowded market warner_lambert com
[ 598 ] gilead hepatitis drug pace become blockbuster sale reach second_quarter testify effectiveness drug essentially cure percent patient side_effect
[ 599 ] roche_holding say new product lift earn roche_holding say today sale rose_percent first quarter fuel revenue testing system new drug include obesity treatment roche whose product include painkiller say
[ 600 ] get market honor promotion marketing campaign tommy inc new york america wash unit company take top honor th_annual award competition sponsor promotion marketing
[ 601 ] pharmaceutical halt study pharmaceutical sell heart cancer treatment halt study recent trial show product because side_effect compete drug set percent decline
[ 602 ] biology concern sell stake research inc mass say sell percent equity stake company green cross corporation japan
[ 603 ] calif

[ 695 ] nad group demand ban artificial hormone cancer risk washington dec health research group demand today federal government remove artificial prescription drug market new information link use development breast cancer
[ 696 ] ignore aid drug shows_promise small test lead aid patient advocate end earth obtain approve drug already market neglect even_though show sign effectiveness delay onset
[ 697 ] panel weigh fate drug cancer federal advisory_panel begin wrestle remove market lung cancer drug fail large clinical_trial
[ 698 ] bottled_water call unsafe people buy bottled_water sure safe tap_water environmental say report issue today sample bottle water sell brand name case group
[ 699 ] rare super_bowl record clash own know recording broadcast want buy tape let sell outside party
[ 700 ] eli_lilly post strong profit drug trial result lift stock first quarter company double sale schizophrenia drug sell antidepressant introduce drug prevent thin bone investor
[ 701 ] lung cancer dru

In [19]:
'''
*Topic 10: regulations
'''
print("amount of documents: ", len(topic_documents_10))
print("search terms unique: ", search_10)#426 documents
print("search terms general: ", search_10b)
for i in range(len(topic_documents_10)):
    print("[",i,"]", " ".join(topic_documents_10[i]))

amount of documents:  213
search terms unique:  ['rule', 'food', 'water']
search terms general:  ['judge', 'smoke', 'order', 'glass', 'smoking']
[ 0 ] spa water bad patron tell cure peril radium discount bath staff guest loss pain note patron report cure drink glass water day say charle talk state health official talk radium content spa water whose suppose pow
[ 1 ] strong warning estrogen label order food drug today order strong health warning label sex hormone take estimate total woman nationwide relieve
[ 2 ] city fight order care mentally_ill city lawyer say yesterday giuliani administration would appeal state judge rule continue mental health care must arrange inmate release rikers_island jail lawyer leonard
[ 3 ] smoke water rainbow add day wimbledon say place doom say even_though hail forecast fire rain evening rainbow main player today wimbledon weather
[ 4 ] president order fast approval cancer drug announce today would take step rule speed cancer drug patient say regulatory c

[ 94 ] say government need combat smoking youth say today think government responsibility discourage smoke young people white_house aide face lobby side say decision make allow food
[ 95 ] portray jail hop mad arrest quite familiar routine complain give glass water prostitute share jail cell advise
[ 96 ] lead rule job safety court agree decide school may abandon court order also agree decide_whether woman bar job may endanger fetus page
[ 97 ] death goal question ask morning oct chain card shop portland_ore prepare tea cookie wife year break handful glass water drink tired
[ 98 ] state order water system reduce radium level whitman administration today order public water system take step remove radium water_supply new testing method find radium level exceed federal standard state official say
[ 99 ] rule let ivax continue sell generic cancer drug say yesterday federal judge allow company keep sell cancer drug taxol rebuff rival company second time october ruling judge
[ 100 ] food mak

[ 189 ] kin lebanon hostage gain court award iran judge washington rule iran pay family rev lawrence martin held_hostage lebanon islamic radical nearly month
[ 190 ] around_nation army order yield file bomb test panel seek question jame earl ray brother improperly oust police judge rule charge reporter mississippi drop feed judge ray rule city improperly dismiss female police employee call investigation allege lesbian activity woman deny lesbian sue city
[ 191 ] smoke choice rule state smoking ban first hit restaurant last year move bar last thursday place resident smoke drink private club exempt ban create opportunity
[ 192 ] video sharing site start award contest aim make hollywood rule judge winner online video sharing site first award contest job short film select first
[ 193 ] clear florida smoker death jury rule today favor decline order cigarette_maker pay family woman smoke year die lung cancer
[ 194 ] food guideline fight cancer panel last week conclude strong enough link diet